In [1]:
import pandas as pd

# Daten aus CSV laden und nach Datumsbereich filtern
data = pd.read_csv('../daten/Gesamtdatensatz.csv')
data = data[(data['timestamp'] >= '2024-04-01') & (data['timestamp'] <= '2024-04-31')]
data

,timestamp,location_id,location_name,ltr_label,rtl_label,weather_condition,temperature,pedestrians_count,unverified,collection_type,...,zone_3_pedestrians_count,zone_3_ltr_pedestrians_count,zone_3_rtl_pedestrians_count,zone_3_adult_pedestrians_count,zone_3_child_pedestrians_count,zone_99_pedestrians_count,zone_99_ltr_pedestrians_count,zone_99_rtl_pedestrians_count,zone_99_adult_pedestrians_count,zone_99_child_pedestrians_count
87848,2024-04-01T00:00:00Z,329,Bahnhofstrasse (Mitte),Hauptbahnhof,Bürkliplatz,cloudy,11.0,97,False,measured,...,65.0,51.0,14.0,65.0,0.0,NaN,NaN,NaN,NaN,NaN
87849,2024-04-01T00:00:00Z,331,Bahnhofstrasse (Nord),Bürkliplatz,Hauptbahnhof,cloudy,11.0,118,False,measured,...,40.0,12.0,28.0,40.0,0.0,NaN,NaN,NaN,NaN,NaN
87850,2024-04-01T00:00:00Z,330,Bahnhofstrasse (Süd),Bürkliplatz,Hauptbahnhof,cloudy,11.0,32,False,measured,...,22.0,8.0,14.0,21.0,1.0,NaN,NaN,NaN,NaN,NaN
87851,2024-04-01T00:00:00Z,670,Lintheschergasse,Hauptbahnhof,Uraniastraße,cloudy,11.0,17,False,measured,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87852,2024-04-01T01:00:00Z,329,Bahnhofstrasse (Mitte),Hauptbahnhof,Bürkliplatz,partly-cloudy-night,10.0,123,False,measured,...,78.0,73.0,5.0,78.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90723,2024-04-30T22:00:00Z,670,Lintheschergasse,Hauptbahnhof,Uraniastraße,partly-cloudy-night,12.8,52,False,measured,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90724,2024-04-30T23:00:00Z,329,Bahnhofstrasse (Mitte),Hauptbahnhof,Bürkliplatz,cloudy,12.7,142,False,measured,...,79.0,50.0,29.0,79.0,0.0,NaN,NaN,NaN,NaN,NaN
90725,2024-04-30T23:00:00Z,331,Bahnhofstrasse (Nord),Bürkliplatz,Hauptbahnhof,cloudy,12.7,123,False,measured,...,61.0,17.0,44.0,61.0,0.0,NaN,NaN,NaN,NaN,NaN
90726,2024-04-30T23:00:00Z,330,Bahnhofstrasse (Süd),Bürkliplatz,Hauptbahnhof,cloudy,12.7,102,False,measured,...,55.0,32.0,23.0,53.0,2.0,NaN,NaN,NaN,NaN,NaN


In [2]:
# Pivot Tabelle mit 
pivot = pd.pivot_table(
    data,
    index='location_name',
    values=['pedestrians_count', 'child_pedestrians_count', 'adult_pedestrians_count'],
    aggfunc='sum',
    fill_value=0
).reset_index()
pivot


,location_name,adult_pedestrians_count,child_pedestrians_count,pedestrians_count
0,Bahnhofstrasse (Mitte),1139624,21499,1161123
1,Bahnhofstrasse (Nord),894808,7313,902121
2,Bahnhofstrasse (Süd),691749,21901,713650
3,Lintheschergasse,326362,3436,329798


In [3]:
# Spalten umbenennen
pivot = pivot.rename(columns={
    'location_name': 'Standorte', 
    'child_pedestrians_count': 'Anzahl Kinder',
    'adult_pedestrians_count': 'Anzahl Erwachsene',
    'pedestrians_count': 'Total',
})
pivot

,Standorte,Anzahl Erwachsene,Anzahl Kinder,Total
0,Bahnhofstrasse (Mitte),1139624,21499,1161123
1,Bahnhofstrasse (Nord),894808,7313,902121
2,Bahnhofstrasse (Süd),691749,21901,713650
3,Lintheschergasse,326362,3436,329798


In [4]:
# Anteil Kinder berechnen
pivot['Anteil Kinder'] = pivot['Anzahl Kinder'] / pivot['Total']
pivot

,Standorte,Anzahl Erwachsene,Anzahl Kinder,Total,Anteil Kinder
0,Bahnhofstrasse (Mitte),1139624,21499,1161123,0.018516
1,Bahnhofstrasse (Nord),894808,7313,902121,0.008106
2,Bahnhofstrasse (Süd),691749,21901,713650,0.030689
3,Lintheschergasse,326362,3436,329798,0.010418


In [ ]:
# Anteil Kinder runden

In [5]:
import altair as alt

bar_width = 80
bar_spacing = 0.5 * bar_width
chart = alt.Chart(pivot).mark_bar(size=bar_width).encode(
    x=alt.X('Standorte').axis(
        labelAngle=-45,
        labelOffset=14,
    ), 
    y=alt.Y('Anteil Kinder:Q').axis(
        format='.1%',
        tickCount=7,
    ),
    tooltip=[
        alt.Tooltip('Anteil Kinder', format='.1%'),
        alt.Tooltip('Anzahl Kinder', format='i'),
        alt.Tooltip('Anzahl Erwachsene', format='i'),
        alt.Tooltip('Total', format='i'),
    ],
).properties(
    width={"step": bar_width + bar_spacing},
    height=400
).configure_axis(
    titleFontSize=14, 
    titleFontWeight="bold",
    labelFontSize=12,
    labelFontWeight="normal",
    labelAngle=0,
).configure_tooltipFormat(

)
chart

alt.Chart(...)

In [6]:
#print(chart.to_json())

import json
# Convert to dict
spec = chart.to_dict()

with open("fokusfrage.json", "w") as f:
    json.dump(spec, f, indent=2)